In [115]:
import string
import re
import pandas as pd
from nltk.stem import PorterStemmer
import numpy as np

# Text preprocessing

Text data often needs preprocessing before analysis can take place. This can account for potential problems like inconsistencies in text formatting, typos and normalisation of punctuation and case.

In the next cell, we introduce some example data for this workshop, a collection of online news articles from October 2021 that use either of the terms `climate change` or `global warming`.

In [11]:
df_news = pd.read_json('data/cc_gw_news_blogs_2021-10-01_2021-10-31.json')

You can investigate the columns of this dataframe to see what metadata is available for these articles, but for the purpose of this workshop, we're most interested in the `title` and `body` columns. They contain the article headline and body text respectively.

In [19]:
df_news.title[0]

'Trudeau says climate progress made at G20, though Canada wanted more ambitious plan - Haida Gwaii Observer'

This example shows one of the cases where you may wish to apply preprocessing to a text, that is the addition of less informative boiler plate text. In this case it is the addition of the source name after the title. We can easily partition the text on a particular string.

In [20]:
df_news.title[0].split(' - ')

['Trudeau says climate progress made at G20, though Canada wanted more ambitious plan',
 'Haida Gwaii Observer']

A word of warning here - be careful if you apply such methods over larger volumes of text or from different sources as they may follow different linguistic styles.

Another common cleaning step in NLP is to standardise the case in a text. Typically, NLP standardises to lowercase text. If this standardisation is not done, then the words `Biden`, `BIDEN`, and `biden` are considered different.

In [21]:
df_news.title[0].lower()

'trudeau says climate progress made at g20, though canada wanted more ambitious plan - haida gwaii observer'

Another aspect of text cleaning you may wish to apply considers the punctuation present in a text. Removing all punctuation from a string is easy using the `string` library. You'll see in the example below that this is not a foolproof step - removal of `'` has contracted `Nicaragua's` to `Nicaraguas`.

In [25]:
print(df_news.title[1])
print(df_news.title[1].translate(str.maketrans('','',string.punctuation)))

Pooling Catastrophe Risk to Protect against Natural Hazards: Nicaragua's Experience in Disaster Risk Management and Finance
Pooling Catastrophe Risk to Protect against Natural Hazards Nicaraguas Experience in Disaster Risk Management and Finance


The particular example listed above highlights another cleaning step you may wish to consider - stemming. Stemming is the process of identifying a word root from various derivative forms. For example, the stemmed forms of `fishes`, `fished` and `fishing` are all `fish`. As a result, you may find stemming a good way to group similar words. You'll see that in this case, the stemmer made all the text lower case automatically.

In [31]:
ps = PorterStemmer()
[ps.stem(w) for w in df_news.title[1].split()]

['pool',
 'catastroph',
 'risk',
 'to',
 'protect',
 'against',
 'natur',
 'hazards:',
 "nicaragua'",
 'experi',
 'in',
 'disast',
 'risk',
 'manag',
 'and',
 'financ']

## A note on encoding

Depending on the source of your text, you may find different text encodings, that is the way that the text is represented digitally. This is usually only problematic in the case of characters like emoji that are not included in the simplest alphabet. If you encounter such errors, try switching to a `utf-8` encoding when you read the file.

## Exercises

Here are a couple of exercises to apply the techniques discussed above.

First, count the number of unique words in the titles of all articles in the dataframe.

In [92]:
## Extract all titles, and make a single long string from them.
titles = ' '.join(df_news.title.values)
## Remove all punctuation.
titles = titles.translate(str.maketrans('','',string.punctuation))
## Apply stemming to the words - note this is optional depending on your personal definition of a word.
titles = [ps.stem(w) for w in titles.split()]
## Converting to a set keeps only unique words, which we then count.
print(f'There are {len(set(titles))} unique words in all titles.')

There are 45391 unique words in all titles.


Which is the most common form of the root word `talk` in the titles? Hint: you may find the `collections` library useful.

In [97]:
import collections
## Extract all titles, and make a single long string from them.
titles = ' '.join(df_news.title.values)
## Remove all punctuation.
titles = titles.translate(str.maketrans('','',string.punctuation))
titles = titles.lower().split()
## Apply stemming to the words.
titles_stemmed = [ps.stem(w) for w in titles]

## Now we have lists of raw and stemmed words.
talk_fulls = []
for full, stem in zip(titles,titles_stemmed):
    if stem == 'talk':
            talk_fulls.append(full)
talk_count = collections.Counter(talk_fulls)
print(talk_count)

Counter({'talks': 2704, 'talk': 744, 'talking': 115, 'talked': 7})


# Text searching

Cleaning the text is designed to make searching and matching parts of the text easier by removing issues that prevent an intended match.

There are a few methods for matching texts in Python. First, we can work on strings directly. Notice the case sensitivity - this is why lower case conversion is important for some NLP tasks.

In [101]:
print(df_news.title[0])
print('Trudeau' in df_news.title[0])
print('trudeau' in df_news.title[0])

Trudeau says climate progress made at G20, though Canada wanted more ambitious plan - Haida Gwaii Observer
True
False


A more powerful means of matching patterns in text uses `regular expressions`. They allow you to define patterns of text to be matched (e.g. any number followed by a bracket such as `1)`) without exhaustively listing the possible combinations. The full power of regex queries are too detailed for this workshop, but I recommend [regex101](https://regex101.com/) as a useful site to define and test queries.

Here's an example query to find GXX groups in texts. Regex queries are normally defined by blocks to match within square brackets `[]` with certain special character groupings predefined. The example below looks for `G` followed by at least one (`+`) digit character (`\d`).

In [103]:
query = r'[G][\d]+'
re.findall(query,df_news.title[0])

['G20']

Applying this over all titles we find the following matches.

In [107]:
g_groups = []
for t in df_news.title:
    g_groups += re.findall(query,t)
print(collections.Counter(g_groups))

Counter({'G20': 3750, 'G7': 101, 'G5': 19, 'G100': 2, 'G3': 2, 'G1': 1, 'G2': 1, 'G02': 1, 'G77': 1})


This example highlights an important caveat when defining text queries - check your data before and after returning the results. Here we were expecting the various levels of international economic fora like `G7` and `G20`. These weren't the only matches in our dataset - some of them are companies or other groups, but they are rare.

## Exercises

Here are a couple of exercises on text matching. Note that these may be achievable with simple text matching, rather than regex but be aware of text inconsistencies.

How many of the headlines mention Justin Trudeau?

In [109]:
## We'll consider a headline as relevant if it mentions just his surname.
count = 0
for t in df_news.title:
    if 'trudeau' in t.lower():
        count += 1
print(f'{count} titles mention Trudeau.')

640 titles mention Trudeau.


How many of the headlines mention one of the G7 countries?

In [117]:
## This query is not an exhaustive list - you could do more to match the UK and US.
query = r'(Canada|Italy|France|Germany|Japan|United Kingdom|UK|United States|US|)'
count = 0
for t in df_news.title:
    if np.any([m != '' for m in re.findall(query,t)]):
        count += 1
print(f'{count} titles mention a G7 country.')

13637 titles mention a G7 country.
